In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rich import print
from aifeel.util import read_corpus
from aifeel.util.preprocess import preprocess_text, preprocess_corpus
from aifeel.util.feature_extraction import extract_features, feature_to_vector
import numpy as np

In [18]:
import dill

# Load the model from the file
with open("export/model/NNClassifier/model.dill", "rb") as f:
    loaded_model = dill.load(f)

with open("export/model/SVM/svm_model.dill", "rb") as f:
    svm_model = dill.load(f)

with open("export/model/NNClassifier/vectorizer.dill", "rb") as f:
    cv = dill.load(f)


In [4]:
negative_words, positive_words = set(read_corpus("negative-words")), set(
    read_corpus("positive-words")
)

In [22]:
def vectorizer(review):
    result = cv.transform([review])
    return result.toarray()[0].tolist() 

In [23]:
def convet_to_vector(clean_corpus):
    features = map(lambda t: extract_features(t, positive_words, negative_words, vectorizer=vectorizer), clean_corpus)
    feature_vectors = list(map(lambda f: feature_to_vector(f, vectorizer=True), features))
    return feature_vectors

In [28]:
def classify_reviews(file_name,model, output_file_name = 'result_testing_challenge.txt'):
    # read challenge file
    reviews = read_corpus(file_name)
    # preprocess text
    clean_reviews = preprocess_corpus(reviews)
    # initailize feature extraction technique
    X = np.array(convet_to_vector(clean_reviews))

    predictions = model.predict(X)
    prob = model.predict_proba(X)
    mapping = {
        0: "Negative",
        1: "Positive"
    }
    
    print(f"Number of sentences: {len(predictions)}")
    with open(output_file_name, 'w') as output_file:
        for prediction, prob in zip(predictions, prob):
            print(f"Result: {mapping[prediction]}\nProbability (Negative, Positive): {prob}")
            output_file.write(f"{prediction}\n")

classify_reviews('testing-challeng', model=loaded_model)

1/1 [==============================] - 0s 34ms/step


Number of sentences: 24

Result: Positive
Probability (Negative, Positive): [4.1633844e-04 9.9958366e-01]

Result: Negative
Probability (Negative, Positive): [9.999217e-01 7.834304e-05]

Result: Positive
Probability (Negative, Positive): [0.16756767 0.8324323 ]

...

Result: Negative
Probability (Negative, Positive): [9.9982619e-01 1.7383647e-04]

Result: Negative
Probability (Negative, Positive): [0.98411876 0.01588124]

Result: Positive
Probability (Negative, Positive): [0.08335251 0.9166475 ]

Result: Negative
Probability (Negative, Positive): [9.9997580e-01 2.4222203e-05]

Result: Positive
Probability (Negative, Positive): [0.2483781 0.7516219]